In [6]:
#imports and constants
import math 
from urllib.request import urlretrieve as rx
EarthRadius = 6378137
MinLatitude = -85.05112878
MaxLatitude = 85.05112878
MinLongitude = -180
MaxLongitude = 180

In [7]:
def Clip(n, minValue, maxValue):
    """
    Clips a number to the specified minimum and maximum values
    input number and its min&max range
    output clipped value in range min and max"""
    return min(max(n,minValue),maxValue)

def MapSize(levelofDetail):
    """
    Determines the map width and height (in pixels) at a specified level
    of detail //map width = map height = 256 * 2pow(level) pixels
    input level of detail 1 to 23
    output returns height=width of map"""
    return int(256 << levelofDetail)

def GroundResolution(latitude,levelofDetail):
    """ 
    Determines the ground resolution (in meters per pixel) at a specified
    latitude and level of detail.
    input latitude,level of detail
    output ground resolution in meters per pixel"""
    latitude=Clip(latitude,MinLatitude,MaxLatitude)
    return math.cos(latitude * math.pi / 180) * 2 * math.pi * EarthRadius / MapSize(levelofDetail)

def MapScale(latitude,levelofDetail,screenDpi):
    """
    Determines the map scale at a specified latitude, level of detail,
    and screen resolution.
    input latitude,levelofDetail,screenDpi
    output mapscale"""
    return GroundResolution(latitude,levelofDetail)*screenDpi/0.0254

def LatLongToPixelXY(latitude,longitude,levelofDetail):
    """
    Converts a point from latitude/longitude WGS-84 coordinates (in degrees)
    into pixel XY coordinates at a specified level of detail.
    input latitude,level of detail,screen dpi
    output pixelX, pixelY"""
    latitude = Clip(latitude, MinLatitude, MaxLatitude)
    longitude = Clip(longitude, MinLongitude, MaxLongitude)
    
    x=(longitude+180)/360
    sinLatitude=math.sin(latitude*math.pi/180)
    y=0.5-math.log10((1 + sinLatitude) / (1 - sinLatitude))/(4*math.pi)
    
    mapSize=MapSize(levelofDetail)
    pixelX=Clip(x*mapSize+0.5,0,mapSize-1)
    pixelY=Clip(y*mapSize+0.5,0,mapSize-1)
    return int(pixelX),int(pixelY)
"""
def PixelXYtoLatLong(pixelX,pixelY,levelofDetail):
    Converts a pixel from pixel XY coordinates at a specified level of detail
    into latitude/longitude WGS-84 coordinates (in degrees).
    input: pixelX,pixelY,level of detail
    output: latitude,longitude
    mapSize=MapSize(levelofDetail)
    x=(Clip(pixelX,0,mapSize-1)/mapSize)-0.5
    y=0.5- (Clip(pixelY,0,mapSize-1)/mapSize)
    
    latitude=90- 360*math.atan(math.exp(-y*2*math.pi))/math.pi
    longitude=360*x
    return latitude,longitude
"""    

def pixelXYToTileXY(pixelX,pixelY):
    """
     Converts pixel XY coordinates into tile XY coordinates of the tile containing
    the specified pixel.
    input :pixelX,pixelY
    output:tileX,tileY
    """
    tileX=pixelX/256
    tileY=pixelY/256
    return int(tileX),int(tileY)

"""   
def TileToPixelXY(tileX,tileY):
    Converts tile XY coordinates into pixel XY coordinates of the upper-left pixel
    of the specified tile.
    input: tileX,tileY
    output: pixelX,pixelY
    pixelX=tileX*256
    pixelY=tileY*256
    return int(pixelX),int(pixelY)
"""

def TileXYToQuadKey(tileX,tileY,levelofDetail):
    """
    Converts tile XY coordinates into a QuadKey at a specified level of detail.
    input:tileX,tileYlevel of detail
    output: quadkey string"""
    quadKey=[]
    for i in range(levelofDetail,0,-1):
        digit=0
        mask=1 <<(i-1)
        if (tileX & mask)!=0:
            digit+=1
        if (tileY & mask)!=0:
            digit+=1
            digit+=1
        quadKey.append(str(digit))
    return "".join(quadKey)

def QuadkeyToTileXY(quadKey):
    """
     Converts a QuadKey into tile XY coordinates.
     input: quadkey
     output: tileX,tileY,levelofDetail"""
    tileX=0
    tileY=0
    levelofDetail=len(quadKey)
    for i in range(levelofDetail,0,-1):
        mask=1 <<(i-1)
        q=quadKey[levelofDetail-i]
        if q=='0':
            break
        elif q=='1':
            tileX |=mask
            break
        elif q=='2':
            tileY |=mask
            break
        elif q=='3':
            tileX |=mask
            tileY |=mask
            break
        else:
            print("invalid quadKey")
            

In [15]:
pixelX,pixelY=LatLongToPixelXY(41,-87,4)
tileX,tileY=pixelXYToTileXY(pixelX,pixelY)
q=TileXYToQuadKey(tileX,tileY,4)
part1_url=" http://h0.ortho.tiles.virtualearth.net/tiles/h"
part2_url=".jpeg?g=131"
request_url=part1_url+q+part2_url
print(request_url)
#rx(part1_url+q+part2_url,"Chicago-23.jpeg")

 http://h0.ortho.tiles.virtualearth.net/tiles/h0322.jpeg?g=131


In [169]:
for i in range(1,24):
    print(i,MapSize(i),GroundResolution(0,i),MapScale(0,i,96))

1 512 78271.51696402048 295829355.4545656
2 1024 39135.75848201024 147914677.7272828
3 2048 19567.87924100512 73957338.8636414
4 4096 9783.93962050256 36978669.4318207
5 8192 4891.96981025128 18489334.71591035
6 16384 2445.98490512564 9244667.357955175
7 32768 1222.99245256282 4622333.678977587
8 65536 611.49622628141 2311166.8394887936
9 131072 305.748113140705 1155583.4197443968
10 262144 152.8740565703525 577791.7098721984
11 524288 76.43702828517625 288895.8549360992
12 1048576 38.21851414258813 144447.9274680496
13 2097152 19.109257071294063 72223.9637340248
14 4194304 9.554628535647032 36111.9818670124
15 8388608 4.777314267823516 18055.9909335062
16 16777216 2.388657133911758 9027.9954667531
17 33554432 1.194328566955879 4513.99773337655
18 67108864 0.5971642834779395 2256.998866688275
19 134217728 0.29858214173896974 1128.4994333441375
20 268435456 0.14929107086948487 564.2497166720688
21 536870912 0.07464553543474244 282.1248583360344
22 1073741824 0.03732276771737122 141.0624